In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for plotting beautiful graphs

# train test split from sklearn
from sklearn.model_selection import train_test_split

# Import Torch 
import torch
import torch.nn as nn
from torchvision import transforms, models,datasets
# from torch.utils.data import SubsetRandomSampler
from torch.autograd import Variable
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
%matplotlib inline

# What's in the current directory?
import os
import pickle
import random

In [2]:
train = pd.read_csv("../input/digit-recognizer/train.csv", dtype=np.float32)
final_test = pd.read_csv("../input/digit-recognizer/test.csv", dtype=np.float32)
sample_sub = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
train.label.head()

0    1.0
1    0.0
2    1.0
3    4.0
4    0.0
Name: label, dtype: float32

In [3]:
train.shape

(42000, 785)

In [4]:
# Seperate the features and labels
targets_np = train.label.values
features_np = train.loc[:, train.columns != 'label'].values/255

# Split into training and test set
features_train, features_test, target_train, target_test = train_test_split(features_np, targets_np, test_size=0.2, random_state=42)


In [5]:
# If a GPU is available, use it
# Pytorch uses an elegant way to keep the code device agnostic
if torch.cuda.is_available():
    device = torch.device("cuda")
    use_cuda = True
else:
    device = torch.device("cpu")
    use_cuda = False
    
print(device)

cpu


In [6]:
# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.from_numpy(features_train)
targetsTrain = torch.from_numpy(target_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
featuresTest = torch.from_numpy(features_test)
targetsTest = torch.from_numpy(target_test).type(torch.LongTensor) # data type is long

In [7]:
batch_size = 256

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = True)

In [8]:
for batch_idx, (inputs, target) in enumerate(train_loader):
    print(inputs.shape)

torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size([256, 784])
torch.Size(

In [9]:
class Classifier(nn.Module):   
    def __init__(self):
        super().__init__()
        # conv block 1
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        
        # conv block 2
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        
        # fully connected layers
        self.fc1 = nn.Linear(64*7*7, 512)
        self.bn5 = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 10)
        
    def forward(self, x):
        # x is [batch_size, channels, heigth, width] = [bs, 1, 28, 28]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2) # x is [bs, 32, 14, 14]
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2) # x is [bs, 64, 7, 7]
        
        x = x.view(x.size(0), -1) # flatten
        
        x = F.relu(self.bn5(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x
    

In [10]:
# Surrogate loss used for training
loss_fn = nn.CrossEntropyLoss()
test_loss_fn = nn.CrossEntropyLoss(reduction='sum')

# spot to save your learning curves, and potentially checkpoint your models
savedir = 'results'
if not os.path.exists(savedir):
    os.makedirs(savedir)

In [11]:
def train(model, train_loader, optimizer, epoch ):
    """Perform one epoch of training."""
    model.train()
    
    for batch_idx, (inputs, target) in enumerate(train_loader):
        inputs, target = inputs.to(device), target.to(device)
        
        # Let them code what's here
        optimizer.zero_grad()
        inputs = Variable(inputs.view(-1,1,28,28))
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        ###
        
        #print('Train Epoch:{}'.format(epoch))
        #if batch_idx % 10 == 0:
            #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            #    epoch, batch_idx * len(inputs), len(train_loader) *len(inputs) ,
            #    100. * batch_idx / len(train_loader), loss.item()))
            

In [12]:
def test(model, test_loader,epoch):
    """Evaluate the model by doing one pass over a dataset"""
    model.eval()
    test_loss = 0
    correct = 0
    test_size = 0
    
    with torch.no_grad():
        for inputs, target in test_loader:
            inputs, target = inputs.to(device), target.to(device)
            
            # TODO: code the evaluation loop
            inputs = Variable(inputs.view(-1,1,28,28))
            output = model(inputs)
            test_size += len(inputs)
            test_loss += test_loss_fn(output, target).item() # sum up batch loss
            # output = batch size * n_classes
            pred = output.argmax(dim=1)
            correct += (pred == target).sum().item()
            # pred = output.max(1, keepdim=True)
            # pred = pred[1] # get the index of the max log-probability

            # correct += pred.eq(target.view_as(pred)).sum().item()
            # #
    test_loss /= test_size
    accuracy = correct / test_size
    print('Test Epoch:{}'.format(epoch))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, test_size,
        100. * accuracy))
    
    return test_loss, accuracy

In [13]:
model = Classifier().to(device)

#lr = 0.0005
optimizer = optim.Adam(model.parameters())

results = {'name':'basic', 'loss': [], 'accuracy':[]}
savefile = os.path.join(savedir, results['name']+'.pkl' )

for epoch in range(1, 50):
    train(model, train_loader, optimizer, epoch)
    loss, acc = test(model, test_loader, epoch)
    
    # save results every epoch
    results['loss'].append(loss)
    results['accuracy'].append(acc)
    with open(savefile, 'wb') as fout:
        pickle.dump(results, fout)

Test Epoch:1

Test set: Average loss: 0.0499, Accuracy: 8278/8400 (98.55%)

Test Epoch:2

Test set: Average loss: 0.0267, Accuracy: 8329/8400 (99.15%)

Test Epoch:3

Test set: Average loss: 0.0320, Accuracy: 8317/8400 (99.01%)

Test Epoch:4

Test set: Average loss: 0.0277, Accuracy: 8320/8400 (99.05%)

Test Epoch:5

Test set: Average loss: 0.0330, Accuracy: 8310/8400 (98.93%)

Test Epoch:6

Test set: Average loss: 0.0286, Accuracy: 8319/8400 (99.04%)

Test Epoch:7

Test set: Average loss: 0.0263, Accuracy: 8325/8400 (99.11%)

Test Epoch:8

Test set: Average loss: 0.0261, Accuracy: 8331/8400 (99.18%)

Test Epoch:9

Test set: Average loss: 0.0261, Accuracy: 8334/8400 (99.21%)

Test Epoch:10

Test set: Average loss: 0.0288, Accuracy: 8329/8400 (99.15%)

Test Epoch:11

Test set: Average loss: 0.0274, Accuracy: 8328/8400 (99.14%)

Test Epoch:12

Test set: Average loss: 0.0220, Accuracy: 8340/8400 (99.29%)

Test Epoch:13

Test set: Average loss: 0.0220, Accuracy: 8348/8400 (99.38%)

Test Epo

In [14]:
final_test_np = final_test.values/255
test_tn = torch.from_numpy(final_test_np)
fake_labels = np.zeros(final_test_np.shape)
fake_labels = torch.from_numpy(fake_labels)

In [15]:
submission_tn_data = torch.utils.data.TensorDataset(test_tn, fake_labels)
submission_loader = torch.utils.data.DataLoader(submission_tn_data, batch_size = 32, shuffle = False)

In [16]:
outputs = []
with torch.no_grad():
    for inputs, target in submission_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = Variable(inputs.view(-1,1,28,28))  
        #inputs = torch.unsqueeze(inputs, 0)
        # TODO: code the evaluation loop
        output = model(inputs)
        pred = output.argmax(dim=1)
        for i in range(len(pred)):
            outputs.append(pred[i].item())

In [17]:
sub =  open('mnist_cnn_sub2.csv','w+')
sub.write('ImageId,Label\n')
for index, prediction in enumerate(outputs):
    sub.write(str(index+1) + ',' + str(prediction) + '\n')
sub.close()